## OpenAI Assistant with Access to Tools

> _We've been merging with tools since the beginning of human evolution, and arguably, that's one of the things that makes us human beings_.
      -- Franklin Foer


**Open AI Assistants**
- Multimodal
- Persistent
- File search
    - Upload upto 10,000 documents
    - Embedding
    - Vector store
    - Page rank
- Python interpreter
- Tools
    - Access external APIs, databases, graphQL
    - Up to 1000 tools
    - Able to use mulitple tools to accomplish tasks


Chat bots are great at returning data from a knoowledge base (RAG), but even more powerful if they can access curated data from an API, query a database to perform operations on the data, or do calculations on the fly with data using the python interpreter.  Importantly, there is no conditional logic in the code, the bot chooses when and how to use the tools.


As a proof of principle I made a blue devil bot.

**Blue Devil Bot**
- File Search
    - Faculty handbook
    - DGS manual
    - DGSA manual
- Python interpreter
- Tools
    - Duke Directory
    - Scholars 
    - Additional tech demonstrations
        - Chinook Music Database (SQL queries)
        - Star Wars SWAPI (GraphQL queries) 






## Demonstration


### World knowledge

In [12]:
user_query("I need a quick and easy recipe for banana pudding, but no vanilla wafers")


assistant > Here's a quick and simple recipe for banana pudding without vanilla wafers:

### Ingredients:
- 3 ripe bananas
- 2 cups milk
- 1/2 cup sugar
- 1/4 cup cornstarch
- 1/4 teaspoon salt
- 2 large eggs
- 1 teaspoon pure vanilla extract
- Whipped cream (optional)

### Directions:
1. **Make the Pudding:**
   - In a saucepan, mix sugar, cornstarch, and salt. Gradually whisk in the milk until smooth.
   - Cook over medium heat, whisking constantly, until the mixture thickens and bubbles, about 7-10 minutes.
   - In a separate bowl, lightly beat the eggs. Gradually whisk about 1/2 cup of the hot milk mixture into the eggs to temper them, so they won’t curdle.
   - Pour the egg mixture back into the saucepan with the rest of the milk mixture, stirring constantly.
   - Cook over medium heat, whisking constantly, for another 2-3 minutes until the pudding is thick and creamy.
   - Remove from heat and stir in the vanilla extract.

2. **Assemble the Pudding:**
   - Slice the bananas into

### Duke directory

In [13]:

user_query("who has the OG username hkm and what is their phone number?")


assistant > function > get_duke_directory_information


assistant > function

{'query': 'hkm'}

assistant > The user with the username "hkm" is Heather MacAlpine. Her phone number is +1 919 684 1620.



### Faculty handbook

In [14]:
user_query("I am married to my laboratory manager, what is the duke policy on nepotism?")


assistant > file_search


assistant > Duke University's policy on nepotism specifies that employees should not be involved in decisions affecting the faculty appointment of their immediate family members, which includes spouses. In exceptional cases where a relative has unique and necessary skills not available in the labor market, a waiver can be sought. Such waivers require a written request and approval from the provost or their designee. If a waiver is approved, further personnel actions affecting the relative also require prior authorization from the individual's department chair, or from the provost if the relative is of the department chair[0].
[0] Duke_faculty_handbook.pdf


### DGS Manual

In [15]:
user_query("How many members of a phd thesis committee can dissent before a student fails a thesis defense at duke?")


assistant > file_search


assistant > Two or more negative votes cast by committee members during a PhD thesis defense at Duke University will result in the student failing the defense. If the committee includes only the required minimum of four members, a single adverse vote fails the candidate. However, if the committee has more than four members, a single negative vote does not fail the candidate unless it is from the professor supervising the dissertation[0].
[0] duke_dgs_manual.pdf


### Python

In [16]:
user_query("Calculate the fist 10 numbers of a Fibonacci sequence using code")


assistant > code_interpreter


assistant > The first 10 numbers of the Fibonacci sequence are: 0, 1, 1, 2, 3, 5, 8, 13, 21, and 34.



### Scholars

In [17]:
# here llm has to use multiple tools - first look up the duid and then query scholars.  
# again no conditional logic, just magic... 


user_query("Papers published by David MacAlpine in 2021")


assistant > function > get_duke_directory_information


assistant > function

{'query': 'David MacAlpine'}

assistant > function > get_scholars_data


assistant > function

{'api_query': '/widgets/api/v0.9/people/publications/all.json', 'duid': '0409422', 'start': '2021-01-01', 'end': '2021-12-31'}

assistant > David MacAlpine published several papers in 2021. Here are some of them:

1. **Cell-Cycle-Dependent Chromatin Dynamics at Replication Origins**
   - Authors: Li Y, Hartemink AJ, MacAlpine DM
   - Published in: Genes (Basel), December 2021
   - DOI: [10.3390/genes12121998](https://doi.org/10.3390/genes12121998)
   - [Link to Publication](https://scholars.duke.edu/individual/pub1503844)

2. **Disruption of origin chromatin structure by helicase activation in the absence of DNA replication**
   - Authors: Hoffman RA, MacAlpine HK, MacAlpine DM
   - Published in: Genes Dev, October 2021
   - DOI: [10.1101/gad.348517.121](https://doi.org/10.1101/gad.348517.121)
   - [Link to Publica

### Querying the chinook music database.  

In [20]:
# here the model generates a sql query from natural language with multiple joins
# against the chinook test database.

user_query("who are the top 5 artists by number of tracks?")


assistant > function > get_music_information


assistant > function

{'query': 'SELECT Artist.Name, COUNT(Track.TrackId) AS TracksCount FROM Artist INNER JOIN Album ON Artist.ArtistId = Album.ArtistId INNER JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Artist.ArtistId ORDER BY TracksCount DESC LIMIT 5'}

assistant > The top 5 artists by number of tracks in the database are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks



## Details and code

In [ ]:
# Install python bits

!pip install openai
!pip install requests


In [1]:
# Import needed modules

import json
import requests
import os
from openai import OpenAI 


In [2]:
# Secret squirrel

api_key = os.getenv("OPENAI_API_KEY")
duke_api_key = os.getenv("DUKE_API_KEY")

# Using latest openAI API that supports streaming
# This is all possible on Azure, but  with the older api
# you have to use active polling and async to keep track of 
# threads and messages -- which is above my coding paygrade

client = OpenAI(api_key=api_key)

In [3]:
# hardcoding the already created assistant id, 
# but is provided on create and updates

DUKE_ASSISTANT_ID = "asst_ENdlDPeBasfiIMMBaneaxBgD"

In [4]:
# helper functions 

def show_json(obj):
    display(json.loads(obj.model_dump_json()))


# main function for interacting with assistant API
def user_query (question):
    event_handler = EventHandler()
    thread = client.beta.threads.create(
    messages=[
         {
            "role": "user",
            "content": question
        }
    ]
    )
 

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=DUKE_ASSISTANT_ID,
        instructions="You are a helpful blue devil concierge. Answer policy and academic questions from the faculty handbook, DGS manual or the dgsa manual",
        event_handler=event_handler,
    ) as stream:
        stream.until_done()





In [ ]:
# Creat the persistent bot

assistant = client.beta.assistants.create(
    name="Blue Devil Bot",
    instructions="You are a personal concierge for Duke University",
    model="gpt-4-turbo",
)

show_json(assistant)

{'id': 'asst_ENdlDPeBasfiIMMBaneaxBgD',
 'created_at': 1714417241,
 'description': None,
 'instructions': 'You are a personal concierge for Duke University',
 'metadata': {},
 'model': 'gpt-4-turbo',
 'name': 'Blue Devil Bot',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0}

In [ ]:
# Setting up the vector store and uploading knowledge

# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Duke Documents")
 
# Ready the files for upload to OpenAI 
file_paths = ["data/Duke_faculty_handbook.pdf","data/duke_dgsa_manual.pdf","data/duke_dgs_manual.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation. 
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)


In [ ]:
show_json(vector_store)

{'id': 'vs_oUmZ02HaQkLuPxBN1QAKJi1L',
 'created_at': 1714417411,
 'file_counts': {'cancelled': 0,
  'completed': 0,
  'failed': 0,
  'in_progress': 0,
  'total': 0},
 'last_active_at': 1714417411,
 'metadata': {},
 'name': 'Duke Documents',
 'object': 'vector_store',
 'status': 'completed',
 'usage_bytes': 0,
 'expires_after': None,
 'expires_at': None}

In [ ]:
# update assistant with file store data

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

In [11]:
# Tool Schemas

# Duke Scholars

duid_swagger =   {
            "name": "get_scholars_data",
            "description": "Look up scholarly activities for faculty at Duke ",
            "parameters": {
                "type": "object",
                "properties": {
                    "api_query": {
                        "type": "string",
                        "description": f"""
                                uri  maps to a duke unique id (duid) which must first be obtained from the duke directory.  
                                count is the number of records to return and format should be json
                                The full swagger schema for the scholars API is provide here:
                                {scholars_swagger_schema}
                                construct and return a fully formed api path in plain text, not in JSON
                                """,
                    },
                    "duid": {
                        "type": "string",
                        "description": "a seven digit unique identifier for an indvidual which must be obtained from the ldap directory"
                    },
                    "start": {
                        "type": "string",
                        "description": "start date in the format YYYY-MM-DD"
                    },
                    "end": {
                        "type": "string",
                        "description": "end date in the format YYYY-MM-DD"
                    },
                },
                "required": ["api_query","duid"],
            },
        }
       

In [10]:
# Duke LADP directory

directory_json = {
            "name": "get_duke_directory_information",
            "description": "Get directory information from the Duke LDAP directory, including phone number, email, and address information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "search query term typically a name, netid, or duid (7 digit identifier)"
                    }
                },
                "required": ["query"]
            },
        }

In [9]:
# Chinook music database

music_json =   {
            "name": "get_music_information",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {chinook_db_schema}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    

In [49]:
# Update the bots with tools and functions

assistant = client.beta.assistants.update(
    assistant_id=DUKE_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"},
        {"type": "function", "function": directory_json},
        {"type": "function", "function": music_json},
        {"type": "function", "function": duid_swagger}
    ],
)
show_json(assistant)

{'id': 'asst_ENdlDPeBasfiIMMBaneaxBgD',
 'created_at': 1714417241,
 'description': None,
 'instructions': 'You are a personal concierge for Duke University',
 'metadata': {},
 'model': 'gpt-4-turbo',
 'name': 'Blue Devil Bot',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search'},
  {'function': {'name': 'get_duke_directory_information',
    'description': 'Get directory information from the Duke LDAP directory, including phone number, email, and address information',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string',
       'description': 'search query term typically a name, netid, or duid (7 digit identifier)'}},
     'required': ['query']}},
   'type': 'function'},
  {'function': {'name': 'get_music_information',
    'description': 'Use this function to answer user questions about music. Input should be a fully formed SQL query.',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string'

In [8]:
# Functions called by "tools" to query external endpoints

def fetch_detailed_info(ldapkey):
    detail_url = f"https://streamer.oit.duke.edu/ldap/people/{ldapkey}"
    params = {'access_token': duke_api_key}
    response = requests.get(detail_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def query_duke_api(query):
    base_url = "https://streamer.oit.duke.edu/ldap/people"
    params = {
        'q': query,
        'access_token': duke_api_key
    }
    
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        results = response.json()
        detailed_results = []

        # Iterate through each result to fetch detailed info using the ldapkey
        for result in results:
            ldapkey = result.get('ldapkey')
            if ldapkey:
                details = fetch_detailed_info(ldapkey)
                if details:
                    detailed_results.append(details)

        return detailed_results
    else:
        return f"Error: Received status code {response.status_code}"



def query_music_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results


def query_scholars_database(arguments):
    api_query=arguments["api_query"]
    

    base_url = f"https://scholars.duke.edu{api_query}"
    params = {
        'uri': arguments["duid"],
        #'access_token': duke_api_key
    }
    if 'start' in arguments:
        params['start'] = arguments['start']

    # Check if 'end' is in arguments and add it to params if it is
    if 'end' in arguments:
        params['end'] = arguments['end']
    
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        results = response.json()
        
        return results
    else:
        return f"Error: Received status code {response.status_code}" 






In [7]:
# Event handling

from typing_extensions import override
from openai import AssistantEventHandler
import sqlite3
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        if tool_call.type == "function":
           print(f"\nassistant > {tool_call.type} > {tool_call.function.name}\n", flush=True)
        #else:
        print(f"\nassistant > {tool_call.type}\n",flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

    @override
    def on_event(self, event):
      # Retrieve events that are denoted with 'requires_action'
      # since these will have our tool_calls
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data, run_id):
      tool_outputs = []
        
      for tool in data.required_action.submit_tool_outputs.tool_calls:
       
        if tool.function.name == "get_duke_directory_information":
          arguments = json.loads(tool.function.arguments)
          print(arguments)
          output = json.dumps(query_duke_api(arguments["query"]))
          tool_outputs.append({"tool_call_id": tool.id, "output": output})
        elif tool.function.name == "get_music_information":
          #print(tool.id)
          conn = sqlite3.connect("data/new_chinook.db")
          #print("Opened database successfully")
          arguments = json.loads(tool.function.arguments)
          print(arguments)
          output = json.dumps(query_music_database(conn,arguments["query"]))
          #print(output)
          tool_outputs.append({"tool_call_id": tool.id, "output": output})
        elif tool.function.name == "get_scholars_data":
          arguments = json.loads(tool.function.arguments)
          print(arguments)
          output = json.dumps(query_scholars_database(arguments))
          tool_outputs.append({"tool_call_id": tool.id, "output": output})
          
      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
         stream.until_done()
        #for text in stream.text_deltas:
        #  print(text, end="", flush=True)
        #  print()






In [1]:
chinook_db_schema = """

CREATE TABLE [Album]
(
    [AlbumId] INTEGER  NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),
    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Artist]
(
    [ArtistId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])
);

CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),
    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Employee]
(
    [EmployeeId] INTEGER  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [FirstName] NVARCHAR(20)  NOT NULL,
    [Title] NVARCHAR(30),
    [ReportsTo] INTEGER,
    [BirthDate] DATETIME,
    [HireDate] DATETIME,
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60),
    CONSTRAINT [PK_Employee] PRIMARY KEY  ([EmployeeId]),
    FOREIGN KEY ([ReportsTo]) REFERENCES [Employee] ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Genre]
(
    [GenreId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Genre] PRIMARY KEY  ([GenreId])
);

CREATE TABLE [Invoice]
(
    [InvoiceId] INTEGER  NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Invoice] PRIMARY KEY  ([InvoiceId]),
    FOREIGN KEY ([CustomerId]) REFERENCES [Customer] ([CustomerId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [InvoiceLine]
(
    [InvoiceLineId] INTEGER  NOT NULL,
    [InvoiceId] INTEGER  NOT NULL,
    [TrackId] INTEGER  NOT NULL,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    [Quantity] INTEGER  NOT NULL,
    CONSTRAINT [PK_InvoiceLine] PRIMARY KEY  ([InvoiceLineId]),
    FOREIGN KEY ([InvoiceId]) REFERENCES [Invoice] ([InvoiceId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([TrackId]) REFERENCES [Track] ([TrackId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [MediaType]
(
    [MediaTypeId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_MediaType] PRIMARY KEY  ([MediaTypeId])
);

CREATE TABLE [Playlist]
(
    [PlaylistId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Playlist] PRIMARY KEY  ([PlaylistId])
);

CREATE TABLE [PlaylistTrack]
(
    [PlaylistId] INTEGER  NOT NULL,
    [TrackId] INTEGER  NOT NULL,
    CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY  ([PlaylistId], [TrackId]),
    FOREIGN KEY ([PlaylistId]) REFERENCES [Playlist] ([PlaylistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([TrackId]) REFERENCES [Track] ([TrackId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Track]
(
    [TrackId] INTEGER  NOT NULL,
    [Name] NVARCHAR(200)  NOT NULL,
    [AlbumId] INTEGER,
    [MediaTypeId] INTEGER  NOT NULL,
    [GenreId] INTEGER,
    [Composer] NVARCHAR(220),
    [Milliseconds] INTEGER  NOT NULL,
    [Bytes] INTEGER,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId]),
    FOREIGN KEY ([AlbumId]) REFERENCES [Album] ([AlbumId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([GenreId]) REFERENCES [Genre] ([GenreId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([MediaTypeId]) REFERENCES [MediaType] ([MediaTypeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);



/*******************************************************************************
   Create Primary Key Unique Indexes
********************************************************************************/

/*******************************************************************************
   Create Foreign Keys
********************************************************************************/
CREATE INDEX [IFK_AlbumArtistId] ON [Album] ([ArtistId]);

CREATE INDEX [IFK_CustomerSupportRepId] ON [Customer] ([SupportRepId]);

CREATE INDEX [IFK_EmployeeReportsTo] ON [Employee] ([ReportsTo]);

CREATE INDEX [IFK_InvoiceCustomerId] ON [Invoice] ([CustomerId]);

CREATE INDEX [IFK_InvoiceLineInvoiceId] ON [InvoiceLine] ([InvoiceId]);

CREATE INDEX [IFK_InvoiceLineTrackId] ON [InvoiceLine] ([TrackId]);

CREATE INDEX [IFK_PlaylistTrackPlaylistId] ON [PlaylistTrack] ([PlaylistId]);

CREATE INDEX [IFK_PlaylistTrackTrackId] ON [PlaylistTrack] ([TrackId]);

CREATE INDEX [IFK_TrackAlbumId] ON [Track] ([AlbumId]);

CREATE INDEX [IFK_TrackGenreId] ON [Track] ([GenreId]);

CREATE INDEX [IFK_TrackMediaTypeId] ON [Track] ([MediaTypeId]);

"""

In [16]:
scholars_swagger_schema = """

{
  "swagger": "2.0",
  "info": {
    "version": "0.9.0",
    "title": "Swagger for Vivo Widgets",
    "description": "Vivo widgets API in the swagger-2.0 specification"
  },
  "produces": [
    "application/json",
    "text/html",
    "application/javascript"
  ],
  "paths": {
    "/widgets/api/v0.9/people/complete/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person or attributes of a person - in a variety of formats",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExtNoHtml"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "$ref": "#/definitions/Person"
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/publications/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's publications",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Publication"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/artistic_works/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's artistic works",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/ArtisticWork"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/awards/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's awards",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Award"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/grants/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's grants",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Grant"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/courses/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's courses",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Course"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/newsfeeds/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's newsfeeds",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Newsfeed"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/professional_activities/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's professional activities",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/ProfessionalActivity"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/positions/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's positions",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Position"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/addresses/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's addresses",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Address"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/overview/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's overview",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Contact"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/people/contact/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns a person's contact information",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          },
          {
            "$ref": "#/parameters/startParam"
          },
          {
            "$ref": "#/parameters/endParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Contact"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/organizations/people/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns an organization (or specific attributes of an organization) - in a variety of formats",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/OrganizationPerson"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/api/v0.9/organizations/grants/{count}.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "Returns an organization (or specific attributes of an organization) - in a variety of formats",
        "parameters": [
          {
            "$ref": "#/parameters/countParam"
          },
          {
            "$ref": "#/parameters/formatParamExt"
          },
          {
            "$ref": "#/parameters/uriParam"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/OrganizationGrant"
              }
            }
          },
          "404": {
            "$ref": "#/responses/NotFound"
          }
        }
      }
    },
    "/widgets/search/modified.{format}": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "search for most recently modified URIs - starting from a specified date",
        "parameters": [
          {
            "name": "since",
            "in": "query",
            "type": "string",
            "format": "date",
            "description": "date in YYYY-MM-DD format"
          },
          {
            "name": "format",
            "type": "string",
            "in": "path",
            "enum": [
              "json"
            ],
            "required": true,
            "description": "format is requested by *.extension - but the only returned format is 'json' at the moment"
          },
          {
            "name": "offset",
            "in": "query",
            "type": "integer",
            "required": false,
            "description": "where to start the list (if there are more than 1000)"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "$ref": "#/definitions/ModifiedList"
            }
          }
        }
      }
    },
    "/widgets/api/v0.9/profile/images": {
      "get": {
        "tags": [
          "widgets"
        ],
        "description": "most recently modified images - starting from a specified date",
        "parameters": [
          {
            "name": "since",
            "in": "query",
            "type": "string",
            "format": "date",
            "description": "date in YYYY-MM-DD format"
          },
          {
            "name": "offset",
            "in": "query",
            "type": "integer",
            "required": false,
            "description": "where to start the list (if there are more than 1000)"
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "schema": {
              "$ref": "#/definitions/ProfileImages"
            }
          }
        }
      }
    }
  },
  "parameters": {
    "countParam": {
      "name": "count",
      "type": "string",
      "in": "path",
      "required": true,
      "description": "how many records to show - 'all' is a valid value",
      "enum": [
        "5",
        "10",
        "15",
        "all"
      ]
    },
    "formatParam": {
      "name": "format",
      "type": "string",
      "description": "In what format to return results ('js' and 'html' are valid, but redirect)",
      "enum": [
        "json",
        "jsonp"
      ],
      "required": true,
      "in": "path"
    },
    "formatParamExt": {
      "name": "format",
      "type": "string",
      "description": "In what format to return results",
      "enum": [
        "json",
        "jsonp",
        "html",
        "js"
      ],
      "required": true,
      "in": "path"
    },
    "formatParamExtNoHtml": {
      "name": "format",
      "type": "string",
      "description": "In what format to return results (html is not supported for this api entry point)",
      "enum": [
        "json",
        "jsonp",
        "js"
      ],
      "required": true,
      "in": "path"
    },
    "uriParam": {
      "name": "uri",
      "type": "string",
      "in": "query",
      "required": true,
      "description": "which URI to look at.  If missing will return 'key not found: uri'.  This can be simply a duid (e.g. Duke Unique Identifier - 7 digits)."
    },
    "formattingParam": {
      "name": "formatting",
      "type": "string",
      "in": "query",
      "required": false,
      "enum": [
        "detailed",
        "abbreviated"
      ]
    },
    "styleParam": {
      "name": "style",
      "type": "string",
      "in": "query",
      "required": false,
      "enum": [
        "styled",
        "unstyled"
      ]
    },
    "startParam": {
      "name": "start",
      "type": "string",
      "in": "query",
      "required": false,
      "description": "start range in yyyy-mm-dd format"
    },
    "endParam": {
      "name": "end",
      "type": "string",
      "in": "query",
      "required": false,
      "description": "end range in yyyy-mm-dd format"
    }
  },
  "responses": {
    "NotFound": {
      "description": "Nothing found to return"
    }
  },
  "definitions": {
    "WidgetItem": {
      "type": "object",
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string"
        },
        "attributes": {
          "type": "object"
        }
      }
    },
    "Organization": {
      "type": "object",
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string"
        },
        "attributes": {
          "type": "object"
        }
      }
    },
    "PersonAttributes": {
      "type": "object",
      "properties": {
        "preferredTitle": {
          "type": "string",
          "description": "The title in the Positions section of the widgets that has the lowest 'rank' value (highest precedence value)"
        },
        "phoneNumber": {
          "type": "string",
          "description": "Phone number from the Duke directory (LDAP)"
        },
        "primaryEmail": {
          "type": "string",
          "description": "Email adddress from the Duke directory (LDAP)"
        },
        "middleName": {
          "type": "string",
          "description": "The person's middle name (if available)"
        },
        "profileURL": {
          "$ref": "#/definitions/uri",
          "description": "The customizable and human-readable URL for the profile.  The default profileURL is based on the Duke email alias"
        },
        "imageModTime": {
          "type": "string",
          "description": "The latest modification date and time of the profile image"
        },
        "suffixName": {
          "type": "string",
          "description": "The suffix in the professional name as specified in Duke's faculty appointment system (dFAc)"
        },
        "imageUri": {
          "$ref": "#/definitions/uri",
          "description": "URI for the page that links to the full profle image and thumbnail"
        },
        "mentorshipAvailabilities": {
          "type": "string",
          "description": "A text description about a person's availability to mentor others at Duke"
        },
        "netid": {
          "type": "string",
          "description": "Duke Net ID"
        },
        "uri": {
          "$ref": "#/definitions/uri",
          "description": "The unique identifier for a person in Scholars@Duke. Faculty URIs include a value that is their hashed Duke Unique ID.  Non-faculty URIs include their Duke Net IDs.  The URI redirects to the URL of the Scholars@Duke profile."
        },
        "alternateId": {
          "type": "string",
          "description": "Duke Unique ID"
        },
        "prefixName": {
          "type": "string",
          "description": "The prefix in the professional name as specified in Duke's faculty appointment system (dFac)"
        },
        "imageDownload": {
          "$ref": "#/definitions/uri",
          "description": "A link to the full-size profile image"
        },
        "lastName": {
          "type": "string",
          "description": "The last name in the professional name as specified in Duke's faculty appointment system (dFac)"
        },
        "firstName": {
          "type": "string",
          "description": "The first name in the professional name as specified in Duke's faculty appointment system (dFac)"
        },
        "imageThumbnailUri": {
          "$ref": "#/definitions/uri",
          "description": "The URI for a page that contains a link to the thumbnail profile image"
        },
        "imageThumbnailDownload": {
          "$ref": "#/definitions/uri",
          "description": "The link to the thumbnail of a profile image"
        },
        "overview": {
          "type": "string",
          "description": "The text description summarizing a person’s research."
        },
        "preferredCitationFormat": {
          "$ref": "#/definitions/uri",
          "description": "The preferred citation style in which publications are displayed on the profile.  This preference is set by the user."
        },
        "imageFileName": {
          "type": "string",
          "description": "The standardized file name for the profile image based off of the person's Scholars URI"
        }
      }
    },
    "Person": {
      "type": "object",
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri",
          "description": "The unique identifier for a person in Scholars@Duke.  Faculty URIs include a value that is their hashed Duke Unique ID.  Non-faculty URIs include their Duke Net IDs.  The URI redirects to the URL of the Scholars@Duke profile."
        },
        "updatedAt": {
          "type": "string",
          "description": "The last modified date for an individual profile. The updatedAt is also returned by an endpoint (https://scholars.duke.edu/widgets/search/modified.json?since=2016-04-15) where you can query all profiles that have changed since a given date. "
        },
        "vivoType": {
          "$ref": "#/definitions/uri",
          "description": "The category of profile determined by the affiliation with Duke.  This is value distinguishes faculty profiles from non-faculty profies "
        },
        "label": {
          "type": "string",
          "description": "The Professional Name (Last, First, Middle) designated in dFac "
        },
        "title": {
          "type": "string",
          "description": "The Preferred title as designated in dFac.  This is the title in which the attribute of 'rank' has the lowest value."
        },
        "publications": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Publication"
          },
          "description": "A list of publications metadata coming from elements.duke.edu"
        },
        "awards": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Award"
          },
          "description": "A list of awards & honors metadata that are manually entered in Scholars@Duke.  Because the data is manually entered, this section may also include grants that were not submitted to Duke’s grants database, SPS"
        },
        "artisticWorks": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/ArtisticWork"
          },
          "description": "A list of non-print scholarly outputs that represent a faculty member’s primary scholarship and are not adequately captured as a publication citation"
        },
        "artisticEvents": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/ArtisticEvent"
          },
          "description": "A list of non-print scholarly outputs that represent a faculty member’s primary scholarship and are not adequately captured as a publication citation"
        },
        "grants": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Grant"
          },
          "description": "A list of select awarded grants from Duke’s Sponsored Project System (SPS). If applicable, one grant uri will represent multiple renewal years, as well as any associated mini grants.  Grants with publicity restrictions are not included."
        },
        "courses": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Course"
          },
          "description": "A list of courses taught in the past 3 years"
        },
        "professionalActivities": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/ProfessionalActivity"
          },
          "description": "A list of manually-entered activites falling into the categories of Service to Duke, Service to the Profession, Outreach & Engaged Scholarsip, or Presentations."
        },
        "positions": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Position"
          },
          "description": "A list of faculty appointments and center memberships captured in dFac. For non-faculty, this section includes the HR job title as well as affiliations entered through Scholars@Duke."
        },
        "addresses": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Address"
          },
          "description": "Both the physical office location (aka External Mailing Address) and the Duke PO Box address (aka Interoffice Address) as entered in Duke@Work"
        },
        "educations": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Education"
          },
          "description": "Awarded degrees and professional training as entered in dFac.  This section is only available for faculty."
        },
        "researchAreas": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/ResearchArea"
          },
          "description": "Areas of expertise using subject headings from MesH, Library of Congress, and a custom Duke University vocabulary."
        },
        "webpages": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Webpage"
          },
          "description": "Web links that were manually entered onto a person’s profile (typically includes links to a CV, department site, personal site, lab page, or Google Scholar page)"
        },
        "geographicalFocus": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/GeographicalFocus",
            "description": "Geographic regions in which a person has expertise, has conducted teaching activities, or has conducted research.  Currently available regions are current countries and US states"
          }
        },
        "newsfeeds": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Newsfeed"
          },
          "description": "A combination of news articles automatically reported by the Office of News & Communications (Duke Today) or manually entered on a Scholars profile. into the profile"
        },
        "academicPositions": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/AcademicPositions"
          },
          "description": "Previous and current academic positions outside of Duke.  This would include visiting professorships.  In Scholars@Duke this section is labeled“Academic Positions Outside Duke."
        },
        "gifts": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Gifts"
          },
          "description": "This section contains competitive funding that is not captured in Duke’s Sponsored Projects System, but rather awarded directly to the PI.  In Scholars@Duke this section is labeled Fellowships, Supported Research, & Other Grants"
        },
        "licenses": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/Licenses"
          },
          "description": "Medical licensure and other credentials not captured in dFac"
        },
        "pastAppointments": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/PastAppointments"
          },
          "description": "Duke appointment history, not including active appointments which are listed separately in the Positions section of the widgets"
        },
        "attributes": {
          "$ref": "#/definitions/PersonAttributes",
          "description": "Additional non-array person attributes such as the overview, pretty profile URL, phone number, and profile picture"
        }
      }
    },
    "Contact": {
      "$ref": "#/definitions/PersonAttributes"
    },
    "OrganizationGrant": {
      "type": "object",
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string"
        },
        "attributes": {
          "type": "object",
          "properties": {
            "organization": {
              "$ref": "#/definitions/uri"
            },
            "organizationName": {
              "type": "string"
            }
          }
        }
      }
    },
    "OrganizationPerson": {
      "type": "object",
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string"
        },
        "attributes": {
          "type": "object",
          "properties": {
            "organization": {
              "$ref": "#/definitions/uri",
              "description": "URI for the organization, built from the organization ID in SAP"
            },
            "organizationName": {
              "type": "string",
              "description": "display name for the organization"
            },
            "position": {
              "$ref": "#/definitions/uri",
              "description": "a URI that describes a person's affiliation with or appointment in an organization"
            },
            "positionType": {
              "$ref": "#/definitions/uri",
              "description": "desribes the type of affiliation or appointment"
            },
            "profileURL": {
              "$ref": "#/definitions/uri",
              "description": "the human-readable URL for a profile"
            },
            "imageModTime": {
              "type": "string",
              "description": "the most recent modification time for the profile image"
            },
            "imageUri": {
              "$ref": "#/definitions/uri",
              "description": "URI for intermediate page that links to full image and thumbnail downloads"
            },
            "imageDownload": {
              "$ref": "#/definitions/uri",
              "description": "link to full profile image"
            },
            "imageThumbnailUri": {
              "$ref": "#/definitions/uri",
              "description": "URI for page that links to thumbnail download"
            },
            "imageThumbnailDownload": {
              "$ref": "#/definitions/uri",
              "description": "link to thumbnail of profile image"
            },
            "overview": {
              "type": "string",
              "description": "the overview paragraph that summarizes a person’s research"
            },
            "imageFileName": {
              "type": "string",
              "description": "standardized name for image that includes the Scholars URI"
            }
          }
        }
      }
    },
    "Publication": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The title of the publication"
        },
        "attributes": {
          "$ref": "#/definitions/PublicationAttributes",
          "description": "The individual metadata elements that make up a citation, as well as 4 pre-formatted citation styles"
        }
      }
    },
    "Award": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "A concatenation of the Award Name & the Award Sponsor"
        },
        "attributes": {
          "$ref": "#/definitions/AwardAttributes",
          "description": "Awards metadata including the award type, award name, award sponsor, and the award date"
        }
      }
    },
    "ArtisticWork": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "Title of the artisic work"
        },
        "attributes": {
          "$ref": "#/definitions/ArtisticWorkAttributes",
          "description": "Metadata about the work, including the abstract, weblink, dates, and roles / collaborators"
        }
      }
    },
    "ArtisticEvent": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "Title of the artisic event"
        },
        "attributes": {
          "$ref": "#/definitions/ArtisticEventAttributes",
          "description": "Metadata about an event"
        }
      }
    },
    "Grant": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The full title of a grant"
        },
        "attributes": {
          "$ref": "#/definitions/GrantAttributes",
          "description": "Metadata about the grant, including the person's role, sponsor, Duke administrating department, and dates"
        }
      }
    },
    "Course": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The course number and title "
        },
        "attributes": {
          "$ref": "#/definitions/CourseAttributes",
          "description": "Details about the course including the person's role"
        }
      }
    },
    "ProfessionalActivity": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "A concatenation of the name of the professional activity and other details found in the attributes section"
        },
        "attributes": {
          "$ref": "#/definitions/ProfessionalActivityAttributes",
          "description": "Metadata about the activity including subcategories, role, description, and dates"
        }
      }
    },
    "Position": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The appointment or affiliation title (for faculty) or HR job title (for non-faculty)"
        },
        "attributes": {
          "$ref": "#/definitions/PositionAttributes",
          "description": "Details about the position including the associated department, school, and title precedence"
        }
      }
    },
    "Address": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "Text for the address"
        },
        "attributes": {
          "$ref": "#/definitions/AddressAttributes",
          "description": "Individual data elements that make up the address label"
        }
      }
    },
    "Education": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "A concatenation of the Awarded Degree (or training) and the year of completion"
        },
        "attributes": {
          "$ref": "#/definitions/EducationAttributes",
          "description": "Education record metadata including the awarded degree, year of completion, and awarding institution"
        }
      }
    },
    "ResearchArea": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The label of a suject heading"
        },
        "attributes": {
          "$ref": "#/definitions/ResearchAreaAttributes",
          "description": "URI of the person associated with the subject heading"
        }
      }
    },
    "Webpage": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "Display label for a manually added web link"
        },
        "attributes": {
          "$ref": "#/definitions/WebpageAttributes",
          "description": "Details about the weblink including the related person and the underlying URL"
        }
      }
    },
    "GeographicalFocus": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The official name of the country or state"
        },
        "attributes": {
          "$ref": "#/definitions/GeographicalFocusAttributes",
          "description": "Details about the geographical focus including the related person and the category of the focus (research, teaching, or expertise)"
        }
      }
    },
    "Newsfeed": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The title of the news article"
        },
        "attributes": {
          "$ref": "#/definitions/NewsfeedAttributes",
          "description": "Details about the news article including related person, date, link to article, and news source"
        }
      }
    },
    "Gifts": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "Name of the competitive funds"
        },
        "attributes": {
          "$ref": "#/definitions/GiftsAttributes",
          "description": "Details about the gift"
        }
      }
    },
    "Licenses": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "A concatenation of the license number, the licensing state, and the license year"
        },
        "attributes": {
          "$ref": "#/definitions/LicensesAttributes",
          "description": "Details about the license"
        }
      }
    },
    "PastAppointments": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The appointment title"
        },
        "attributes": {
          "$ref": "#/definitions/PastAppointmentsAttributes",
          "description": "Details about the appointment"
        }
      }
    },
    "AcademicPositions": {
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string",
          "description": "The title of the news article"
        },
        "attributes": {
          "$ref": "#/definitions/AcademicPositionsAttributes",
          "description": "Details about the position"
        }
      }
    },
    "PublicationAttributes": {
      "type": "object",
      "properties": {
        "icmjeCitation": {
          "type": "string",
          "description": "The publication citation in the ICMJE format"
        },
        "mlaCitation": {
          "type": "string",
          "description": "The publication citation in the MLA format "
        },
        "nlmCitation": {
          "type": "string",
          "description": "The publication citation in the NLM format "
        },
        "isFavorite": {
          "type": "boolean",
          "description": "A flag indicating whether a publication was marked as 'favorite' in elements.duke.edu"
        },
        "authorList": {
          "type": "string",
          "description": "A list of all authors as 'Last Name, First Initial'"
        },
        "dateTime": {
          "$ref": "#/definitions/uri"
        },
        "conferenceLocation": {
          "type": "string",
          "description": "Location where a conference paper was presented"
        },
        "abstract": {
          "type": "string",
          "description": "Publication abstract"
        },
        "pmid": {
          "type": "string",
          "description": "The PubMed ID only available if the citation source is from PubMed or European PubMed"
        },
        "apaCitation": {
          "type": "string",
          "description": "The publication citation in the APA format"
        },
        "year": {
          "type": "string",
          "description": "The publication date as 'YYYY-MM-DDT00:00:00'"
        },
        "endPage": {
          "type": "integer",
          "description": "The end page for an article or book section"
        },
        "publicationSource": {
          "type": "string",
          "description": "The source of the publication record.  Metadata will vary slightly between the various sources.  The default source is PubMed or European PubMed."
        },
        "chicagoCitation": {
          "type": "string",
          "description": "The publication citation in the Chicago format"
        },
        "authorshipType": {
          "$ref": "#/definitions/uri"
        },
        "issue": {
          "type": "string",
          "description": "The journal issue"
        },
        "subtypes": {
          "type": "string",
          "description": "While the vivoType generally represents major publishing formats, the subtype relays the genre and/or content of the publication"
        },
        "publicationVenue": {
          "type": "string",
          "description": "The journal represented as unlinked text (when the ISSN is not identified) or a URI (when the ISSN is identified)"
        },
        "publishedIn": {
          "type": "string",
          "description": "The publisher name"
        },
        "startPage": {
          "type": "integer",
          "description": "The start page of an article or book section"
        },
        "doi": {
          "type": "string",
          "description": "The digital object identifier"
        },
        "volume": {
          "type": "string",
          "description": "The journal volume"
        }
      }
    },
    "AwardAttributes": {
      "type": "object",
      "properties": {
        "serviceType": {
          "type": "string",
          "description": "Represents the level of award.  Values can be “Department”, “International”, “National”, “Other”, “School, “State”, or “University”"
        },
        "name": {
          "type": "string",
          "description": "The name of the award"
        },
        "datePrecision": {
          "$ref": "#/definitions/uri"
        },
        "awardedBy": {
          "type": "string",
          "description": "The name of the awarding body or sponsor"
        },
        "awardedByUri": {
          "$ref": "#/definitions/uri"
        },
        "date": {
          "type": "string",
          "description": "The award date"
        }
      }
    },
    "ArtisticWorkAttributes": {
      "type": "object",
      "properties": {
        "role": {
          "type": "string",
          "description": "One or many roles that a person had on the work"
        },
        "abstract": {
          "type": "string",
          "description": "A description of the work itself.  Collaborators will share the same abstract"
        },
        "role_description": {
          "type": "string",
          "description": "A description providing more details on a person's role"
        },
        "date": {
          "type": "string",
          "description": "The creation date of the work as 'YYYY-MM-DDT00'"
        },
        "type_description": {
          "type": "string",
          "description": "One or more cateogires tagging the type of work "
        },
        "date_precision": {
          "$ref": "#/definitions/uri"
        },
        "collaborators": {
          "type": "string",
          "description": "A list of all manually-entered Duke and non-Duke collaborators."
        }
      }
    },
    "ArtisticEventAttributes": {
      "type": "object",
      "properties": {
        "venue": {
          "type": "string",
          "description": "venue"
        },
        "description": {
          "type": "string",
          "description": "event description"
        },
        "startYear": {
          "type": "integer",
          "description": ""
        },
        "endYear": {
          "type": "integer",
          "description": ""
        }
      }
    },
    "GrantAttributes": {
      "type": "object",
      "properties": {
        "endDate": {
          "type": "string",
          "description": "The project period end date for the most recent proposal in a series"
        },
        "roleName": {
          "type": "string",
          "description": "A person’s role on the grant"
        },
        "awardedBy": {
          "type": "string",
          "description": "The name of the grant sponsor"
        },
        "administeredBy": {
          "type": "string",
          "description": "The name of the Duke department that received the grant"
        },
        "awardedByUri": {
          "$ref": "#/definitions/uri"
        },
        "administeredByUri": {
          "$ref": "#/definitions/uri"
        },
        "startDate": {
          "type": "string",
          "description": "The project period start date for the first proposal in a series"
        }
      }
    },
    "CourseAttributes": {
      "type": "object",
      "properties": {
        "role": {
          "$ref": "#/definitions/uri"
        },
        "roleName": {
          "type": "string",
          "description": "Currently, this field is a duplication to the course label (concatenation of the course name and course title)"
        }
      }
    },
    "ProfessionalActivityAttributes": {
      "type": "object",
      "properties": {
        "startDatePrecision": {
          "$ref": "#/definitions/uri"
        },
        "serviceOrEventName": {
          "type": "string",
          "description": "The title of the activity"
        },
        "serviceType": {
          "type": "string",
          "description": "The subcategory for the activity"
        },
        "role": {
          "type": "string"
        },
        "hostOrganization": {
          "type": "string",
          "description": "The name of the organization that hosted the activity"
        },
        "locationOrVenue": {
          "type": "string",
          "description": "The name of the event venue or geogrpahical location for the activity"
        },
        "startDate": {
          "type": "string",
          "description": "The start date of the activity as 'YYYY-MM-DDT00'"
        }
      }
    },
    "AddressAttributes": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string"
        },
        "state": {
          "type": "string"
        },
        "postalCode": {
          "type": "string"
        },
        "personUri": {
          "$ref": "#/definitions/uri"
        },
        "address1": {
          "type": "string"
        }
      }
    },
    "EducationAttributes": {
      "type": "object",
      "properties": {
        "degreeUri": {
          "$ref": "#/definitions/uri"
        },
        "endDate": {
          "type": "string"
        },
        "degree": {
          "type": "string"
        },
        "organizationUri": {
          "$ref": "#/definitions/uri"
        },
        "institution": {
          "type": "string"
        },
        "dateTimeUri": {
          "$ref": "#/definitions/uri"
        },
        "personUri": {
          "$ref": "#/definitions/uri"
        },
        "endUri": {
          "$ref": "#/definitions/uri"
        }
      }
    },
    "ResearchAreaAttributes": {
      "type": "object",
      "properties": {
        "personUri": {
          "$ref": "#/definitions/uri"
        }
      }
    },
    "WebpageAttributes": {
      "type": "object",
      "properties": {
        "personUri": {
          "$ref": "#/definitions/uri"
        },
        "linkURI": {
          "$ref": "#/definitions/uri"
        }
      }
    },
    "GeographicalFocusAttributes": {
      "type": "object",
      "properties": {
        "focusTypeLabel": {
          "type": "string",
          "description": "The category of relationship between a person andna geographic region"
        },
        "focusType": {
          "$ref": "#/definitions/uri"
        },
        "personUri": {
          "$ref": "#/definitions/uri"
        }
      }
    },
    "NewsfeedAttributes": {
      "type": "object",
      "properties": {
        "newsDatetimeObj": {
          "$ref": "#/definitions/uri"
        },
        "newsLink": {
          "$ref": "#/definitions/uri"
        },
        "newsDatetime": {
          "type": "string"
        },
        "newsMonth": {
          "type": "integer"
        },
        "newsDay": {
          "type": "integer"
        },
        "newsYear": {
          "type": "integer"
        },
        "newsSource": {
          "type": "string"
        }
      }
    },
    "GiftsAttributes": {
      "type": "object",
      "properties": {
        "dateTimeEnd": {
          "type": "string"
        },
        "dateTimeEndDay": {
          "type": "integer"
        },
        "dateTimeEndMonth": {
          "type": "integer"
        },
        "dateTimeEndYear": {
          "type": "integer"
        },
        "dateTimeEndValue": {
          "$ref": "#/definitions/uri"
        },
        "dateTimeInterval": {
          "$ref": "#/definitions/uri"
        },
        "dateTimeStart": {
          "type": "string"
        },
        "dateTimeStartDay": {
          "type": "integer"
        },
        "dateTimeStartMonth": {
          "type": "integer"
        },
        "dateTimeStartYear": {
          "type": "integer"
        },
        "dateTimeStartValue": {
          "$ref": "#/definitions/uri"
        },
        "description": {
          "type": "string"
        },
        "donor": {
          "type": "string"
        },
        "giftAmount": {
          "type": "string"
        },
        "role": {
          "type": "string"
        }
      }
    },
    "LicensesAttributes": {
      "type": "object",
      "properties": {
        "datetime": {
          "$ref": "#/definitions/uri"
        },
        "number": {
          "type": "string"
        },
        "state": {
          "type": "string"
        },
        "year": {
          "type": "integer"
        }
      }
    },
    "PastAppointmentsAttributes": {
      "type": "object",
      "properties": {
        "dateUri": {
          "$ref": "#/definitions/uri"
        },
        "endDateiimeUri": {
          "type": "string"
        },
        "endYear": {
          "type": "integer"
        },
        "organizationLabel": {
          "type": "string"
        },
        "schoolLabel": {
          "type": "string"
        },
        "schoolUri": {
          "$ref": "#/definitions/uri"
        },
        "startDateiimeUri": {
          "type": "string"
        },
        "startYear": {
          "type": "integer"
        }
      }
    },
    "AcademicPositionsAttributes": {
      "type": "object",
      "properties": {
        "institute": {
          "type": "string"
        },
        "roles": {
          "type": "string"
        },
        "startDate": {
          "type": "string"
        },
        "endDate": {
          "type": "string"
        }
      }
    },
    "PositionAttributes": {
      "type": "object",
      "properties": {
        "uri": {
          "$ref": "#/definitions/uri"
        },
        "vivoType": {
          "$ref": "#/definitions/uri"
        },
        "label": {
          "type": "string"
        },
        "attributes": {
          "type": "object",
          "properties": {
            "schoolLabel": {
              "type": "string"
            },
            "organizationLabel": {
              "type": "string"
            },
            "organizationUri": {
              "$ref": "#/definitions/uri"
            },
            "rank": {
              "type": "integer"
            },
            "personUri": {
              "$ref": "#/definitions/uri"
            },
            "schoolUri": {
              "$ref": "#/definitions/uri"
            }
          }
        }
      }
    },
    "SearchResults": {
      "type": "object",
      "properties": {
        "numFound": {
          "type": "integer"
        },
        "groups": {
          "type": "object"
        },
        "items": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/SearchResultItem"
          }
        }
      }
    },
    "ProfileImages": {
      "type": "object",
      "properties": {
        "since": {
          "type": "string"
        },
        "numFound": {
          "type": "integer"
        },
        "offset": {
          "type": "integer"
        },
        "items": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/ProfileImagesItems"
          }
        }
      }
    },
    "ProfileImagesItems": {
      "type": "object",
      "properties": {
        "duid": {
          "type": "string"
        },
        "lastUpdated": {
          "type": "string"
        },
        "profile": {
          "$ref": "#/definitions/ProfileImagesLinks"
        }
      }
    },
    "ProfileImagesLinks": {
      "type": "object",
      "properties": {
        "profileImageLarge": {
          "type": "string"
        },
        "profileImageMedium":  {
          "type": "string"
        },
        "profileImageSmall":  {
          "type": "string"
        }
      }
    },
    "ModifiedList": {
      "type": "object",
      "properties": {
        "numFound": {
          "type": "integer"
        },
        "offset": {
          "type": "integer"
        },
        "items": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/ModifiedListItem"
          }
        }
      }
    },
    "SearchResultItem": {
      "type": "object"
    },
    "ModifiedListItem": {
      "$ref": "#/definitions/uri"
    },
    "uri": {
      "type": "string"
    }
  }
}







"""

In [6]:
scholars_swagger_schema = """


{
  "swagger": "2.0",
  "info": {
    "title": "Swagger for Vivo Widgets",
    "version": "0.9.0",
    "description": "API for querying people, publications, and grants."
  },
  "produces": [
    "application/json"
  ],
  "paths": {
    "/widgets/api/v0.9/people/complete/{count}.{format}": {
      "get": {
        "summary": "Retrieve detailed information about people.",
        "parameters": [
          {
            "name": "count",
            "in": "path",
            "required": true,
            "type": "string",
            "description": "Number of records to return, 'all' is valid."
          },
          {
            "name": "format",
            "in": "path",
            "required": true,
            "type": "string",
            "description": "Response format (json, jsonp, js)."
          },
          {
            "name": "uri",
            "in": "query",
            "required": true,
            "type": "string",
            "description": "URI of the person."
          },
          {
            "name": "start",
            "in": "query",
            "type": "string",
            "description": "Start of date range for query."
          },
          {
            "name": "end",
            "in": "query",
            "type": "string",
            "description": "End of date range for query."
          }
        ],
        "responses": {
          "200": {
            "description": "Successful retrieval of person details."
          },
          "404": {
            "description": "Person not found."
          }
        }
      }
    },
    "/widgets/api/v0.9/people/publications/{count}.{format}": {
      "get": {
        "summary": "Retrieve a person's publications.",
        "parameters": [
          {
            "name": "count",
            "in": "path",
            "required": true,
            "type": "string",
            "description": "Number of records to return."
          },
          {
            "name": "format",
            "in": "path",
            "required": true,
            "type": "string",
            "description": "Response format (includes json, jsonp, html, js)."
          },
          {
            "name": "uri",
            "in": "query",
            "required": true,
            "type": "string",
            "description": "URI of the person whose publications are queried."
          },
          {
            "name": "start",
            "in": "query",
            "type": "string",
            "description": "Start of date range for query."
          },
          {
            "name": "end",
            "in": "query",
            "type": "string",
            "description": "End of date range for query."
          }
        ],
        "responses": {
          "200": {
            "description": "Successful retrieval of publications."
          },
          "404": {
            "description": "Publications not found."
          }
        }
      }
    },
    "/widgets/api/v0.9/people/grants/{count}.{format}": {
      "get": {
        "summary": "Retrieve a person's grants.",
        "parameters": [
          {
            "name": "count",
            "in": "path",
            "required": true,
            "type": "string",
            "description": "Number of records to return."
          },
          {
            "name": "format",
            "in": "path",
            "required": true,
            "type": "string",
            "description": "Response format (json, jsonp, html, js)."
          },
          {
            "name": "uri",
            "in": "query",
            "required": true,
            "type": "string",
            "description": "URI of the person whose grants are queried."
          },
          {
            "name": "start",
            "in": "query",
            "type": "string",
            "description": "Start of date range for query."
          },
          {
            "name": "end",
            "in": "query",
            "type": "string",
            "description": "End of date range for query."
          }
        ],
        "responses": {
          "200": {
            "description": "Successful retrieval of grants."
          },
          "404": {
            "description": "Grants not found."
          }
        }
      }
    }
  }
}

"""

In [5]:
chinook_db_schema = """

Table: Album
Columns: AlbumId, Title, ArtistId
Table: Artist
Columns: ArtistId, Name
Table: Customer
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: Employee
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: Genre
Columns: GenreId, Name
Table: Invoice
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: InvoiceLine
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: MediaType
Columns: MediaTypeId, Name
Table: Playlist
Columns: PlaylistId, Name
Table: PlaylistTrack
Columns: PlaylistId, TrackId
Table: Track
Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice

"""